# Naive Bayesian Classifier
### Q1. Bayes Rule을 이해하고 Naive  Bayes classifier가 사용하는 사후 확률 계산 과정을 서술하세요.

- Bayes Rule:   
$P(w_i|x) = \frac{P(x|w_i)|P(w_i)}{P(x)} = \frac{P(x|w_i) P(w_i)}{\Sigma_j P(x|w_j)P(w_j)}$
  -
  - $P(w_i|x)\text{: 사후 확률, posterior}\\
P(x|w_i) \text{: 가능도/우도, likelihood}\\
P(w_i) \text{: 사전 확률, prior}\\
P(x) \text{: 증거, evidence}$

A1.Naive Bayes Classifier는 각 특징이 서로 독립적이다라는 나이브한 가정하에 이루어진다.
<br>주어진 입력 벡터 x에 대해 각 클래스 𝑤_𝑖 의 사후 확률 𝑃(𝑤_𝑖∣𝑥)을 계산하고, 이 중 가장 큰 값을 가지는 클래스를 예측 결과로 선택한다. 이러한 과정은 독립성 가정을 통해 계산을 단순화하고, 빠르고 효율적인 분류를 가능하게 하다.


### Q2. Naive Bayes Classification 방법을 이용해서 다음 생성된 리뷰 데이터에 기반한 감정 분석을 해봅시다.

In [1]:
# pip install pandas
import pandas as pd
import re

In [2]:
# 리뷰 데이터 생성
data = {
    'review': [
        'I love this great product! It exceeded my expectations.',
        'The Worst purchase I have ever made. Completely useless.',
        'It is an average product, nothing special but not terrible either.',
        'Great service and who can help but love this design? Highly recommend!',
        'Terrible experience, I will never buy from this poor brand again.',
        'It’s acceptable, but I expected better service, not just an acceptable one.',
        'Absolutely wonderful! I am very satisfied with this great service.',
        'The quality is poor and it broke after one use. Terrible enough!',
        'Acceptable product for the price, but there are better options out there.',
        'Great quality and fast shipping with wonderful service! I love it'
    ],
    'sentiment': [
        'positive', 'negative', 'neutral', 'positive', 'negative',
        'neutral', 'positive', 'negative', 'neutral', 'positive',
    ]
}
df = pd.DataFrame(data)
df.head()

,review,sentiment
0,I love this great product! It exceeded my expe...,positive
1,The Worst purchase I have ever made. Completel...,negative
2,"It is an average product, nothing special but ...",neutral
3,Great service and who can help but love this d...,positive
4,"Terrible experience, I will never buy from thi...",negative


In [3]:
# 불용어 리스트 정의
stopwords = ['i', 'my', 'am', 'this', 'it', 'its', 'an', 'a', 'the', 'is', 'are', 'and', 'product', 'service']

In [4]:
# 텍스트 전처리 함수 정의
def preprocess_text(text):
    # 소문자로 변환
    text = text.lower()
    # 특수 기호 제거
    text = re.sub(r'[^a-z\s]', '', text)
    # 불용어 제거
    words = text.split()
    filtered_words = [word for word in words if word not in stopwords]
    return ' '.join(filtered_words)

# 모든 리뷰에 대해 전처리 수행
df['review'] = df['review'].apply(preprocess_text)

기본적인 데이터 전처리가 완료되었습니다!
이제부터 직접 나이브 베이지안 분류를 수행해 봅시다.  
우리가 분류하고자 하는 문장은 총 두가지 입니다.  
전처리가 완료되었다고 치고,   
첫번째 문장은 **'love, great, awesome'**,  
두번째 문장은 **'terrible, not, never'** 입니다.

사전 확률 $P(positive), P(negative), P(neutral)$을 구합니다.

In [5]:
# 감정의 빈도수 계산
sentiment_counts = df['sentiment'].value_counts()

# 전체 리뷰 수
total_reviews = len(df)

# 사전 확률 계산
P_positive = sentiment_counts['positive'] / total_reviews
P_negative = sentiment_counts['negative'] / total_reviews
P_neutral = sentiment_counts['neutral'] / total_reviews

# 사전 확률 출력
print(f"P(positive): {P_positive:.4f}")
print(f"P(negative): {P_negative:.4f}")
print(f"P(neutral): {P_neutral:.4f}")

P(positive): 0.4000
P(negative): 0.3000
P(neutral): 0.3000


가능도를 구하기 위한 확률들을 계산합니다.  
예: 첫번째 문장 분류를 위해서는, $P(love|positive), P(great|positive), P(awesome|positive)\\
P(love|negative), P(great|negative), P(awesome|negative)\\
P(love|neutral), P(great|neutral), P(great|neutral)$를 구합니다.

이 때 CountVectorizer를 사용하여 도출한 단어 벡터를 활용하면 확률들을 간편하게 구할 수 있습니다.  
참고: https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html

In [6]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
review_array = vectorizer.fit_transform(df['review']).toarray()
review_array

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0,
        0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
        0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
        0, 0, 0, 1, 0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
        1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
        0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
        1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
        1, 0, 0, 0, 0, 0, 1, 0, 0, 0],
       [0, 2, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 

In [7]:
vectorizer.get_feature_names_out()

array(['absolutely', 'acceptable', 'after', 'again', 'average', 'better',
       'brand', 'broke', 'but', 'buy', 'can', 'completely', 'design',
       'either', 'enough', 'ever', 'exceeded', 'expectations', 'expected',
       'experience', 'fast', 'for', 'from', 'great', 'have', 'help',
       'highly', 'just', 'love', 'made', 'never', 'not', 'nothing', 'one',
       'options', 'out', 'poor', 'price', 'purchase', 'quality',
       'recommend', 'satisfied', 'shipping', 'special', 'terrible',
       'there', 'use', 'useless', 'very', 'who', 'will', 'with',
       'wonderful', 'worst'], dtype=object)

In [8]:
vectorizer.vocabulary_

{'love': 28,
 'great': 23,
 'exceeded': 16,
 'expectations': 17,
 'worst': 53,
 'purchase': 38,
 'have': 24,
 'ever': 15,
 'made': 29,
 'completely': 11,
 'useless': 47,
 'average': 4,
 'nothing': 32,
 'special': 43,
 'but': 8,
 'not': 31,
 'terrible': 44,
 'either': 13,
 'who': 49,
 'can': 10,
 'help': 25,
 'design': 12,
 'highly': 26,
 'recommend': 40,
 'experience': 19,
 'will': 50,
 'never': 30,
 'buy': 9,
 'from': 22,
 'poor': 36,
 'brand': 6,
 'again': 3,
 'acceptable': 1,
 'expected': 18,
 'better': 5,
 'just': 27,
 'one': 33,
 'absolutely': 0,
 'wonderful': 52,
 'very': 48,
 'satisfied': 41,
 'with': 51,
 'quality': 39,
 'broke': 7,
 'after': 2,
 'use': 46,
 'enough': 14,
 'for': 21,
 'price': 37,
 'there': 45,
 'options': 34,
 'out': 35,
 'fast': 20,
 'shipping': 42}

In [9]:
frequency_matrix = pd.DataFrame(review_array, columns = vectorizer.get_feature_names_out())
frequency_matrix = pd.concat([df['sentiment'], frequency_matrix], axis=1)
frequency_matrix

,sentiment,absolutely,acceptable,after,again,average,better,brand,broke,but,...,terrible,there,use,useless,very,who,will,with,wonderful,worst
0,positive,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,negative,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1
2,neutral,0,0,0,0,1,0,0,0,1,...,1,0,0,0,0,0,0,0,0,0
3,positive,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0
4,negative,0,0,0,1,0,0,1,0,0,...,1,0,0,0,0,0,1,0,0,0
5,neutral,0,2,0,0,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0
6,positive,1,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,1,1,0
7,negative,0,0,1,0,0,0,0,1,0,...,1,0,1,0,0,0,0,0,0,0
8,neutral,0,1,0,0,0,1,0,0,1,...,0,2,0,0,0,0,0,0,0,0
9,positive,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,0


In [10]:
# 각 감정에 따른 단어 빈도 합계 계산
sentiment_word_totals = frequency_matrix.groupby('sentiment').sum()

# 전체 단어 빈도 합계 계산
total_word_counts = sentiment_word_totals.sum(axis=1)

# 조건부 확률 계산 함수 정의
def calculate_conditional_prob(word, sentiment):
    if word in sentiment_word_totals.columns:
        # P(word | sentiment) = (해당 단어의 감정 내 빈도 + 1) / (해당 감정의 총 단어 빈도 + 단어 종류 수)
        return (sentiment_word_totals.at[sentiment, word] + 1) / (total_word_counts[sentiment] + len(vectorizer.get_feature_names_out()))
    else:
        # 단어가 사전에 없는 경우, 작은 확률 반환 (라플라스 스무딩)
        return 1 / (total_word_counts[sentiment] + len(vectorizer.get_feature_names_out()))

words = ['love', 'great', 'awesome']
sentiments = ['positive', 'negative', 'neutral']

for sentiment in sentiments:
    print(f"Conditional probabilities for {sentiment}:")
    for word in words:
        prob = calculate_conditional_prob(word, sentiment)
        print(f"P({word} | {sentiment}) = {prob:.4f}")
    print("\n")

Conditional probabilities for positive:
P(love | positive) = 0.0500
P(great | positive) = 0.0625
P(awesome | positive) = 0.0125


Conditional probabilities for negative:
P(love | negative) = 0.0128
P(great | negative) = 0.0128
P(awesome | negative) = 0.0128


Conditional probabilities for neutral:
P(love | neutral) = 0.0128
P(great | neutral) = 0.0128
P(awesome | neutral) = 0.0128




독립성 가정을 이용하여 가능도(likelihood)를 구합니다.  
첫번째 문장 예시: $P(love, great, awesome|positive), P(love, great, awesome|negative), P(love, great, awesome|neutral)$

In [11]:
# 각 감정에 대한 가능도를 저장할 딕셔너리
likelihoods = {'positive': [], 'negative': [], 'neutral': []}

# 첫 번째 문장과 두 번째 문장에 포함된 단어
target_review1 = ['love', 'great', 'awesome']
target_review2 = ['terrible', 'not', 'never']

for word in target_review1:
    for sentiment in likelihoods.keys():
        likelihoods[sentiment].append(calculate_conditional_prob(word, sentiment))

print("target_review1")
for sentiment, probs in likelihoods.items():
    print(f"Likelihoods for '{sentiment}': {probs}")

likelihoods = {'positive': [], 'negative': [], 'neutral': []}
for word in target_review2:
    for sentiment in likelihoods.keys():
        likelihoods[sentiment].append(calculate_conditional_prob(word, sentiment))

print("\ntarget_review2")
for sentiment, probs in likelihoods.items():
    print(f"Likelihoods for '{sentiment}': {probs}")

target_review1
Likelihoods for 'positive': [0.05, 0.0625, 0.0125]
Likelihoods for 'negative': [0.01282051282051282, 0.01282051282051282, 0.01282051282051282]
Likelihoods for 'neutral': [0.01282051282051282, 0.01282051282051282, 0.01282051282051282]

target_review2
Likelihoods for 'positive': [0.0125, 0.0125, 0.0125]
Likelihoods for 'negative': [0.038461538461538464, 0.01282051282051282, 0.02564102564102564]
Likelihoods for 'neutral': [0.02564102564102564, 0.038461538461538464, 0.01282051282051282]


위에서 구한 사전 확률과 가능도를 이용하여 타겟 문장이 positive, negative, neutral일 확률을 구하고 최종적으로 어떤 감성일지 분석해봅니다.

In [12]:
import numpy as np

# 사전 확률
P_positive = 0.4
P_negative = 0.3
P_neutral = 0.3

# 첫 번째 문장에 대한 확률 계산
P_target1_positive = P_positive * np.prod([calculate_conditional_prob(word, 'positive') for word in target_review1])
P_target1_negative = P_negative * np.prod([calculate_conditional_prob(word, 'negative') for word in target_review1])
P_target1_neutral = P_neutral * np.prod([calculate_conditional_prob(word, 'neutral') for word in target_review1])

print(f"P(positive|target_review1): {P_target1_positive}")
print(f"P(negative|target_review1): {P_target1_negative}")
print(f"P(neutral|target_review1): {P_target1_neutral}")

# 두 번째 문장에 대한 확률 계산
P_target2_positive = P_positive * np.prod([calculate_conditional_prob(word, 'positive') for word in target_review2])
P_target2_negative = P_negative * np.prod([calculate_conditional_prob(word, 'negative') for word in target_review2])
P_target2_neutral = P_neutral * np.prod([calculate_conditional_prob(word, 'neutral') for word in target_review2])

print(f"\nP(positive|target_review2): {P_target2_positive}")
print(f"P(negative|target_review2): {P_target2_negative}")
print(f"P(neutral|target_review2): {P_target2_neutral}")


P(positive|target_review1): 1.5625000000000004e-05
P(negative|target_review1): 6.321751883882061e-07
P(neutral|target_review1): 6.321751883882061e-07

P(positive|target_review2): 7.812500000000002e-07
P(negative|target_review2): 3.7930511303292367e-06
P(neutral|target_review2): 3.7930511303292367e-06


A2-1.   
Target review1의 분류 결과:

P(positive|target_review1): 1.5625e-05

P(negative|target_review1): 6.32e-07

P(neutral|target_review1): 6.32e-07

-> Target review1은 positive로 분류됩니다. positive 확률이 다른 감정들보다 월등히 높기 때문이다.

Target review2의 분류 결과:

P(positive|target_review2): 7.81e-07

P(negative|target_review2): 3.79e-06

P(neutral|target_review2): 3.79e-06

-> Target review2는 negative 혹은 neutral로 분류됩니다. negative와 neutral의 확률이 동일하게 나왔고, positive보다 훨씬 높기 때문이다.

Q2-2. 나이브 베이지안 기반 확률을 구하는 과정에서 어떤 문제점을 발견할 수 있었나요? 그리고 그 문제를 해결하기 위한 방법에 대해 간략하게 조사 및 서술해 주세요. (힌트: Laplace smoothing)

A2-2.

나이브 베이즈 기반 확률을 구하는 과정에서 발생할 수 있는 주요 문제점은 확률이 0이 되는 문제이다. 특히, 학습 데이터에 특정 단어가 나타나지 않았거나, 특정 감정과 관련된 빈도가 0인 경우, 그 단어의 조건부 확률이 0이 되어 전체 확률도 0이 되어버리는 문제가 발생할 수 있다. 이러한 문제는 모델이 해당 감정에 대한 판단을 할 수 없게 만들어, 잘못된 분류를 초래할 수 있다.

이 문제를 해결하기 위한 대표적인 방법이 라플라스 스무딩(Laplace Smoothing)이다. 라플라스 스무딩은 각 조건부 확률을 계산할 때, 빈도 수가 0인 경우에도 확률이 0이 되는 것을 방지하기 위해 모든 빈도 수에 일정한 값을 더해주는 방법이다.